In [76]:
%%capture
!pip install folium matplotlib mapclassify

In [77]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

COLS_TO_KEEP=[
              'geometry',
              'PARITARIA',]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import NILs

In [78]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


#Import primary schools gdf

In [79]:
ps = gpd.read_file(os.path.join(FOLDER_PATH, 'primary_schools.txt'))
ps = ps[COLS_TO_KEEP]
ps

,geometry,PARITARIA
0,POINT (9.22648 45.50634),No
1,POINT (9.17935 45.45587),SI
2,POINT (9.17328 45.45885),No
3,POINT (9.16963 45.46344),SI
4,POINT (9.16977 45.46558),SI
...,...,...
213,POINT (9.15355 45.48649),SI
214,POINT (9.20132 45.47996),SI
215,POINT (9.22282 45.49158),No
216,POINT (9.18888 45.44230),SI


# Space Join

In [80]:
primary_schools = nil.sjoin(ps,how='left').drop('index_right',axis=1)
primary_schools = primary_schools.rename(columns={'PARITARIA':'public'})
primary_schools['public'] = primary_schools['public'].fillna('No')
primary_schools

,ID_NIL,NIL,area_sqkm,Residenti,geometry,public
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",SI
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",No
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",No
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",No
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ...",No
...,...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ...",SI
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",No
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ...",No
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ...",No


# Groupby

In [91]:
primary_schools_grouped = primary_schools.groupby(['ID_NIL','NIL','area_sqkm','Residenti','public']).size().reset_index(name='count')
primary_schools_grouped['schools_per_sqkm'] = primary_schools_grouped['count']/primary_schools_grouped['area_sqkm']
primary_schools_grouped

,ID_NIL,NIL,area_sqkm,Residenti,public,count,schools_per_sqkm
0,1,duomo,2.341616,16608,No,3,1.281167
1,1,duomo,2.341616,16608,SI,1,0.427056
2,10,stazione centrale - ponte seveso,1.556019,18410,No,2,1.285332
3,10,stazione centrale - ponte seveso,1.556019,18410,SI,3,1.927997
4,11,isola,1.322887,22641,No,1,0.755922
...,...,...,...,...,...,...,...
121,85,parco delle abbazie,13.733841,364,No,1,0.072813
122,86,parco dei navigli,3.623149,354,No,1,0.276003
123,87,assiano,5.840942,211,No,1,0.171205
124,88,parco bosco in citta',7.834006,690,No,1,0.127649


# Pivot

In [94]:
primary_schools_pivot = primary_schools_grouped.pivot(index=['ID_NIL','NIL','area_sqkm','Residenti'], columns=['public'], values=['count','schools_per_sqkm']).fillna(0).reset_index()
primary_schools_pivot = primary_schools_pivot.rename(columns={'No':'private_schools', 'SI':'public_schools'})
primary_schools_pivot


ID_NIL                               NIL  area_sqkm Residenti  \
public                                                                 
0           1                             duomo   2.341616     16608   
1          10  stazione centrale - ponte seveso   1.556019     18410   
2          11                             isola   1.322887     22641   
3          12           maciachini - maggiolina   1.674918     26097   
4          13                   greco - segnano   1.768603     16137   
..        ...                               ...        ...       ...   
83         85               parco delle abbazie  13.733841       364   
84         86                 parco dei navigli   3.623149       354   
85         87                           assiano   5.840942       211   
86         88             parco bosco in citta'   7.834006       690   
87          9     porta garibaldi - porta nuova   0.785638      5921   

                 count                schools_per_sqkm                 
public private_schools public_schools  private_schools public_schools  
0                  3.0            1.0         1.281167       0.427056  
1                  2.0            3.0         1.285332       1.927997  
2                  1.0            1.0         0.755922       0.755922  
3                  3.0            0.0         1.791133       0.000000  
4                  2.0            0.0         1.130836       0.000000  
..                 ...            ...              ...            ...  
83                 1.0            0.0         0.072813       0.000000  
84                 1.0            0.0         0.276003       0.000000  
85                 1.0            0.0         0.171205       0.000000  
86                 1.0            0.0         0.127649       0.000000  
87                 1.0            0.0         1.272850       0.000000  

[88 rows x 8 columns]

In [96]:
# prompt: get the columns to one layer in primary_schools_pivot

primary_schools_pivot.columns = [''.join(col) for col in primary_schools_pivot.columns]
primary_schools_pivot = primary_schools_pivot.rename(columns={'countprivate_schools':'private_schools',
                                                              'countpublic_schools':'public_schools',
                                                              'schools_per_sqkmprivate_schools':'private_schools_per_sqkm',
                                                              'schools_per_sqkmpublic_schools':'public_schools_per_sqkm'})
primary_schools_pivot

,ID_NIL,NIL,area_sqkm,Residenti,private_schools,public_schools,private_schools_per_sqkm,public_schools_per_sqkm
0,1,duomo,2.341616,16608,3.0,1.0,1.281167,0.427056
1,10,stazione centrale - ponte seveso,1.556019,18410,2.0,3.0,1.285332,1.927997
2,11,isola,1.322887,22641,1.0,1.0,0.755922,0.755922
3,12,maciachini - maggiolina,1.674918,26097,3.0,0.0,1.791133,0.000000
4,13,greco - segnano,1.768603,16137,2.0,0.0,1.130836,0.000000
...,...,...,...,...,...,...,...,...
83,85,parco delle abbazie,13.733841,364,1.0,0.0,0.072813,0.000000
84,86,parco dei navigli,3.623149,354,1.0,0.0,0.276003,0.000000
85,87,assiano,5.840942,211,1.0,0.0,0.171205,0.000000
86,88,parco bosco in citta',7.834006,690,1.0,0.0,0.127649,0.000000


In [97]:
#primary_schools_pivot.to_csv(os.path.join(FOLDER_PATH, 'Processed','primary_schools.csv'),index=False)